<a href="https://colab.research.google.com/github/hmghaly/km/blob/main/b2web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')
cwd="/content/drive/MyDrive/KM"
os.chdir(cwd)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


#1- Get the crawl data for dmain and country
save it to the corresponding directory

In [ ]:
#1- First, run common crawl query on the most recent crawl, for the domain required
import os, sys

#https://index.commoncrawl.org/CC-MAIN-2021-10/
#/CC-MAIN-2021-25 - June 2021
!chmod +x utils/cdx-index-client.py
#!./cdx-index-client.py -c CC-MAIN-2021-10 *.io --show-num-pages
#!./cdx-index-client.py -c CC-MAIN-2021-10 *.gov.au --show-num-pages
#!./cdx-index-client.py -c CC-MAIN-2021-10 *.edu.au --fl url -z -d edu.au
#!./cdx-index-client.py -c CC-MAIN-2021-10 *.org.au --fl url -z -d org.au
crawl_version="CC-MAIN-2021-25"
crawl_dir="crawl"
run_dir="aug21" #aug21_au
urls_dir="urls" #<<<<<< add a subdirectry to contain only the urls

#domain="edu.au"
for domain in ["edu.au","com.au","net.au","gov.au","org.au"]:
#for domain in ["org.au"]:
  print("working on domain", domain)
  print("number of pages")
  os.system("utils/cdx-index-client.py -c %s *.%s --show-num-pages"%(crawl_version,domain))
  #full_dir_path=os.path.join(crawl_dir,run_dir,domain)
  full_dir_path=os.path.join(crawl_dir,run_dir,domain,urls_dir) #putting the crawled data into the url subdir
  if not os.path.exists(full_dir_path): os.makedirs(full_dir_path)
  os.system("utils/cdx-index-client.py -c %s *.%s --fl url -z -d %s"%(crawl_version,domain,full_dir_path))
#!./cdx-index-client.py -c CC-MAIN-2021-10 *.com.au --fl url -z -d com.au

working on domain edu.au
number of pages
working on domain com.au
number of pages


In [ ]:
#2- Then extract the list of urls from the results of the query (stored in gzip files) into text files 
import gzip, os, sys
#!ls
sys.path.append("utils")
from utils.extraction_utils import *

crawl_dir="crawl"
run_dir="aug21" #aug21_au
urls_dir="urls" #<<<<<< add a subdirectry to contain only the urls

main_dir_path=os.path.join(crawl_dir,run_dir)
counter_dict={}

for cur_dir in os.listdir(main_dir_path):
  full_dpath=os.path.join(main_dir_path,cur_dir)
  if not os.path.isdir(full_dpath): continue
  url_counter=0
  print(full_dpath)
  out_txt_fname=cur_dir+".txt"
  out_txt_fpath=os.path.join(main_dir_path,out_txt_fname)
  print("output file:", out_txt_fpath)
  list_fopen=open(out_txt_fpath,"w")
  files=os.listdir(full_dpath)
  print(len(files))
  for i,fname in enumerate(files):
    used_urls=[]
    if i%100==0: print(cur_dir, i, "out of", len(files))
    #print(">>>>>>> fname",fname)
    fpath=os.path.join(full_dpath,fname)
    tmp_fopen=open(fpath)
    for tf in tmp_fopen:
      #print([tf])
      #print(tf.split())
      
      cur_url=tf.split()[-1].strip('"{}')
      # print(cur_url)
      if cur_url.split(".")[-1].lower() in ["pdf","png","js","css","jpg"]: continue
      if cur_url.endswith("robots.txt") or cur_url.endswith("wp-admin"): continue
      if "ezproxy" in cur_url: continue
        # print(">>>>>>> URL:", cur_url)
    # gz_open=gzip.open(fpath,'r')
    
    # for gz in gz_open:
    #   #print(str(gz))
    #   cur_url=str(gz)[2:-3]
      bare_url=get_bare_url(cur_url)
      # print(bare_url)
      if bare_url in used_urls: continue
      used_urls.append(bare_url)

      #print(bare_url,len(bare_url.split(".")))
      
      if "com" in cur_dir and len(bare_url.split("."))>3: continue #skip subdomains for .com websites
      if "net" in cur_dir and len(bare_url.split("."))>3: continue #skip subdomains for .net websites
      new_url=cur_url.split("//")[0]+"//"+bare_url  

      url_counter+=1
      #print(">>>>", new_url)
      list_fopen.write(new_url+"\n")
  list_fopen.close()
  counter_dict[cur_dir]=url_counter

for key,val in counter_dict.items():
  print(key, val)


crawl/aug21/edu.au
output file: crawl/aug21/edu.au.txt
126
edu.au 0 out of 126
edu.au 100 out of 126
crawl/aug21/com.au
output file: crawl/aug21/com.au.txt
1670
com.au 0 out of 1670
com.au 100 out of 1670
com.au 200 out of 1670
com.au 300 out of 1670
com.au 400 out of 1670
com.au 500 out of 1670
com.au 600 out of 1670
com.au 700 out of 1670
com.au 800 out of 1670
com.au 900 out of 1670
com.au 1000 out of 1670
com.au 1100 out of 1670
com.au 1200 out of 1670
com.au 1300 out of 1670
com.au 1400 out of 1670
com.au 1500 out of 1670
com.au 1600 out of 1670
crawl/aug21/net.au
output file: crawl/aug21/net.au.txt
45
net.au 0 out of 45
crawl/aug21/gov.au
output file: crawl/aug21/gov.au.txt
129
gov.au 0 out of 129
gov.au 100 out of 129
crawl/aug21/org.au
output file: crawl/aug21/org.au.txt
102
org.au 0 out of 102
org.au 100 out of 102
edu.au 14807
com.au 482865
net.au 16369
gov.au 7519
org.au 32026


In [ ]:
#Now combining all the text files with url lists into one big list
import shutil
crawl_dir="crawl"
run_dir="aug21" #aug21_au
main_dir_path=os.path.join(crawl_dir,run_dir)
txt_files=[os.path.join(main_dir_path,f0) for f0 in os.listdir(main_dir_path) if f0.endswith(".txt")]
out_fpath=os.path.join(main_dir_path,"all_urls.txt")
print(txt_files)
print(out_fpath)
with open(out_fpath,'wb') as wfd:
    for f in txt_files:
        with open(f,'rb') as fd:
            shutil.copyfileobj(fd, wfd)

# !cat "edu.au.txt" "com.au.txt" "gov.au.txt" "org.au.txt" > "all_au_urls.txt"

['crawl/aug21/edu.au.txt', 'crawl/aug21/com.au.txt', 'crawl/aug21/net.au.txt', 'crawl/aug21/gov.au.txt', 'crawl/aug21/org.au.txt']
crawl/aug21/all_urls.txt


#2-Cache the crawled websites

In [ ]:
#Now run the caching script on the full data with 60 workers and a batch of 10000 - we need to get the script to accept arguments
!chmod +x utils/caching_utils.py
!utils/caching_utils.py #with 10000 batch - 60 workers

number of urls to process: 553586
start_i 370000 end_i 553586 n_urls 553586
current i 370000 time: 0.0
current i 380000 time: 535.51
current i 390000 time: 545.66
current i 400000 time: 552.7
current i 410000 time: 530.58
current i 420000 time: 539.38
current i 430000 time: 552.73
current i 440000 time: 551.85
tcmalloc: large alloc 1442463744 bytes == 0x555bdef22000 @  0x7f0802ac01e7 0x555bb5d01488 0x555bb5ccd945 0x555bb5da0fb3 0x555bb5da08db 0x555bb5ccefcc 0x555bb5dc0c0d 0x555bb5d430d8 0x555bb5d3dc35 0x555bb5cd073a 0x555bb5d42f40 0x555bb5cd065a 0x555bb5d42f40 0x555bb5cd065a 0x555bb5d3ed67 0x555bb5cd065a 0x555bb5d3ed67 0x555bb5cd065a 0x555bb5d3ed67 0x555bb5cd065a 0x555bb5d42f40 0x555bb5c0fd14 0x555bb5dc0d21 0x555bb5dee869 0x555bb5d3ec77 0x555bb5c0fd14 0x555bb5dc0d21 0x555bb5dee869 0x555bb5d3ec77 0x555bb5cd0b99 0x555bb5d11bc9
current i 450000 time: 847.66
/usr/local/lib/python3.7/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probabl

In [ ]:
!wc -l crawl/aug21/cached_all.txt
!tail crawl/aug21/cached_all.txt

!wc -l crawl/aug21/all_urls.txt
!tail crawl/aug21/all_urls.txt
#357290 crawl/aug21/cached_all.txt
!wc -l crawl/aug21/classified_all.txt
!tail crawl/aug21/classified_all.txt

553031 crawl/aug21/cached_all.txt
http://nsw.cfmeu.org.au<br>CFMEU NSW | Building better lives for more than 30,000 construction workers.<br>CFMEU NSW | Building better lives for more than 30,000 construction workers. Skip to main content Search Search<br>Your Union FAQ<br>Message from the Executive<br>Team Members<br>Campaigns<br>Members Pay your dues<br>Change your details<br>RDOs<br>Wages Information<br>Member Benefits<br>Health & Safety Safety Alerts / Guidance<br>Asbestos Exposure Form<br>Delegate / HSR Registration<br>Notification of Concern / Incident<br>OHS Reps Information<br>Provisional Improvement Notice (PIN) WHS Act 2011 (NSW)<br>News Centre<br>Shop<br>Contact<br>Join CFMEU<br>Stay Informed Name * Email * Postcode * Submit Fast Find<br>2021 RDOs<br>JOIN<br>PAY DUES / UPDATE DETAILS<br>WAGES INFO<br>Affiliates<br>News<br>News: 02 August 2021<br>Financial Statements for the CFMEU for year ended 31 March 2021<br>News: 02 August 2021<br>OFFICER AND RELATED PARTY DISCLOSURE STA

#3- Now do the classification

In [ ]:
#Step 1 - load the Word2Vec model
# #from gensim.models import Doc2Vec
# import gensim.downloader as api
# #wv = api.load('word2vec-google-news-300', return_path=True)
# model = api.load("word2vec-google-news-300")  # download the model and return as object ready for use
# word_vectors = model.wv #load the vectors from the model
# model.save("google-news.wv")
#Now loading the model
from gensim.models import KeyedVectors
wv_model=KeyedVectors.load("google-news.wv",mmap='r')


In [ ]:
#Get the classification structure and save it as json
from utils.classification_utils import *
xl_fname='290721.xlsx'
cat_struct_obj=cat_struct(xl_fname)
cat_struct_obj.save("data_dict_new.json")
#cat_struct_obj.data_dict

{'child_dict': {'accounting': [('accounting-services',
    'Accounting & Bookkeeping')],
  'audit-compliance': [('financial-audit-compliance-services', 'Financial'),
   ('fraud-incident-investigation-services', 'Fraud Investigation'),
   ('environmental-consulting-audit-services', 'Environmental'),
   ('safety-audit-compliance-services', 'Workplace Safety'),
   ('financial-audit-compliance-services', 'Audit & Compliance')],
  'banking-financial-investment-services': [('investment-finance-advisory-services',
    'Financial'),
   ('investment-finance-advisory-services', 'Financial Consulting'),
   ('investment-fund-management-services', 'Investment'),
   ('lending-services', 'Lending')],
  'clothing-products': [('workwear', 'Workwear')],
  'construction': [('paint-coating', 'Paints'),
   ('commercial-industrial-heating-cooling',
    'Commercial & Industrial Heating & Cooling'),
   ('commercial-construction-services', 'Commercial Construction'),
   ('residential-construction-renovation-se

In [ ]:
#Run the classification script
!chmod +x utils/classification_utils.py
!utils/classification_utils.py 
#Time for classification: 35624.586965084076 #without any optimization

started to load model
loaded model 9.649028062820435
loaded classification structure -4.935264587402344e-05
utils/classification_utils.py:100: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if self.text_vec==[]: return
5000 time: 294.91
10000 time: 280.02
15000 time: 285.3
20000 time: 596.35
25000 time: 335.51
35000 time: 650.1
40000 time: 321.09
45000 time: 323.5
50000 time: 308.05
55000 time: 322.46
60000 time: 336.36
65000 time: 327.67
70000 time: 321.09
75000 time: 329.97
85000 time: 661.38
90000 time: 323.59
95000 time: 325.59
100000 time: 331.92
110000 time: 632.56
115000 time: 315.42
130000 time: 974.08
135000 time: 334.42
140000 time: 311.64
145000 time: 313.16
150000 time: 326.29
155000 time: 324.31
160000 time: 333.73
165000 time: 321.03
170000 time: 312.93
175000 time: 311.12
180000 time: 313.71
185000 time: 309.91
190000 time: 335.14
195000 time: 318.36
200000 time: 314.7
205000 time: 317.4
210000 time: 330.71
215000 time: 325.9

In [ ]:
#Try to use pypy to make it faster
!pypy3 utils/classification_utils.py 

Traceback (most recent call last):
  File "utils/classification_utils.py", line 2, in <module>
    from gensim.models import KeyedVectors
ModuleNotFoundError: No module named 'gensim'


#4- Final grouping, sorting and analysis of the classified data

In [7]:
#now analyzing the classified websites
#let's split the data classified into com/gov/edu/org directories
#now analyzing the classified websites

import json, os
from itertools import groupby
from utils.classification_utils import *
from utils.extraction_utils import *


json_fopen=open("data_dict_new.json")
data_dict_json=json.loads(json_fopen.read())
json_fopen.close()
id_dict=data_dict_json["id_dict"]


country_code="au"
run_code="aug21"
run_dir=run_code #aug21_au <<<<<<<<<<

crawl_dir="crawl"
classified_fname="classified_all.txt"
#classified_fpath="crawl/aug21/classified_all.txt"
classified_fpath=os.path.join(crawl_dir,run_dir,classified_fname)
out_dir="classified"
out_dir_path=os.path.join(crawl_dir,run_dir,out_dir)
#out_dir="results26apr"
if not os.path.exists(out_dir_path): os.makedirs(out_dir_path)

only_top_category=False #select only one category per website
n_top=3 #number of top categories

json_fopen=open("data_dict.json")
data_dict_json=json.loads(json_fopen.read())
json_fopen.close()
id_dict=data_dict_json["id_dict"]

classified_fopen=open(classified_fpath)
info_dict={}
all_pred_list=[]
used_urls=[]
for i_,f in enumerate(classified_fopen):
  #if i_<120000: continue
  #if i_>120000: break
  if i_%50000==0: print(i_)
  f_split=f.strip("\n\r\t").split("\t")
  if len(f_split)!=2: continue
  url,info_str=f_split
  url=url.split("<br>")[0]
  bare_url=get_bare_url(url)
  url_type=bare_url.split(".")[-2].lower()
  if url_type=="net": url_type="com" #<<<<<<<<<<<<<<<<<<<<< COMBINING .net with .com
  # if bare_url in used_urls: continue
  # used_urls.append(bare_url)
  if url_type=="com" and len(bare_url.split("."))>3: 
    #print(">>>>>>>>>>>>>>",bare_url)
    continue
  #print(bare_url,url_type)

  # url_no_http=url.split("//")[-1]
  # url_split=[v for v in url_no_http.split(".") if v!="www"]
  #if len(url_split)>3: continue
  #print(url_split)


  local_info_dict=json.loads(info_str)
  info_dict[bare_url]=local_info_dict
  top_preds=local_info_dict["top_preds"]
  if not top_preds: continue
  #print(top_preds)
  #print(url)
  if only_top_category: all_pred_list.append((top_preds[0][0],url,top_preds[0][1]))
  else:
    for tp in top_preds[:n_top]:
      #print(tp[0],tp[1])
      all_pred_list.append((url_type,tp[0],url,tp[1]))

  #print(f_split)
  #print("---")
classified_fopen.close()

all_pred_list.sort(key=lambda x:x[0])
grouped=[(key,[v[1:] for v in group]) for key,group in groupby(all_pred_list,lambda x:x[0])]
for k, grp in grouped: #k is the domain name
  print(k, len(grp))
  grp.sort(key=lambda x:x[0])
  grouped2=[(key,[v[1:] for v in group]) for key,group in groupby(grp,lambda x:x[0])]
  subdir_path=os.path.join(out_dir_path,k) #for .com .gov ... etc directories under the main directory
  if not os.path.exists(subdir_path): os.makedirs(subdir_path)
  for cat_id, grp2 in grouped2:
    cat_fpath=os.path.join(subdir_path,cat_id+".txt")
    cat_fopen=open(cat_fpath,"w")
    used_urls=[]
    grp2.sort(key=lambda x:-x[-1])
    print(cat_id, len(grp2))
    for g2 in grp2:
      url0,score0=g2
      bare_url0=get_bare_url(url0)
      # if bare_url0 in used_urls: continue
      # used_urls.append(bare_url0)
      local_info_obj={}
      cur_info=info_dict[bare_url0]
      top_words=cur_info["top_words"]

      top_preds=cur_info["top_preds"]
      #just_preds=["%s:%s"%(v[0],v[1]) for v in top_preds]
      just_preds=[(v[0],v[1]) for v in top_preds]
      #print(top_preds)
      cur_cat_words=[v[-1] for v in top_preds if v[0]==cat_id]
      if cur_cat_words: cur_cat_words=cur_cat_words[0] 
      local_info_obj["preds"]=just_preds
      local_info_obj["top_words"]=top_words
      local_info_obj["cat_words"]=cur_cat_words
      local_info_json=json.dumps(local_info_obj)

      line="%s\t%s\t%s\n"%(url0,score0,local_info_json)
      cat_fopen.write(line)
      #print(g2)
  # for g in grp[:10]:
  #   print(g)
    #print("------")
  cat_fopen.close()
  print("==============")

print("finished creating classified files for each domain, now zipping")
#zip_fname="%s-%s1.zip"%(country_code,run_code)
zip_fname="%s-%s"%(country_code,run_code)
zip_fpath=os.path.join(crawl_dir,run_dir,zip_fname)
# zip_command='zip -r %s %s'%(zip_fpath,out_dir_path)
# os.system(zip_command)
import shutil
shutil.make_archive(zip_fpath, 'zip', out_dir_path)

#!zip -r classified-au-aug21-2.zip classified2/

0
50000
100000
150000
200000
250000
300000
350000
400000
450000
au 3
3d-printing 1
animal-transport-logistics 1
manufacture-of-metallic-components-structures 1
com 1280934
3d-printing 49672
accounting-services 6375
advertising-services 7144
animal-nutrition-health-products 11139
animal-transport-logistics 54881
animal-veterinary-services 62308
architecture-design-services 12456
artistic-design-services 22314
asset-property-inspection-verification 3701
biomedical-pharmaceutical-research 3422
building-property-insurance 16069
building-property-maintenance-services 12521
buses-vans 8939
business-insurance 5412
catering 16944
clinic-care-services 15933
commercial-construction-services 9816
commercial-industrial-heating-cooling 8497
content-writing-editing 22021
data-analysis-services 7489
data-management-warehousing-services 4468
demolition-services 5091
earth-moving-equipment 5380
electric-maintenance-contracting-services 5069
electrical-engineering-services 1709
electronic-engineering-su

'/content/drive/My Drive/KM/crawl/aug21/au-aug21.zip'

In [4]:
import os
run_dir="aug21" #aug21_au <<<<<<<<<<
country_code="au"
!zip -r classified-au-aug21-2.zip classified2/

  adding: classified2/ (stored 0%)
  adding: classified2/au/ (stored 0%)
  adding: classified2/au/3d-printing.txt (deflated 37%)
  adding: classified2/au/animal-transport-logistics.txt (deflated 39%)
  adding: classified2/au/manufacture-of-metallic-components-structures.txt (deflated 37%)
  adding: classified2/com/ (stored 0%)
  adding: classified2/com/3d-printing.txt (deflated 86%)
  adding: classified2/com/accounting-services.txt (deflated 90%)
  adding: classified2/com/advertising-services.txt (deflated 88%)
  adding: classified2/com/animal-nutrition-health-products.txt (deflated 87%)
  adding: classified2/com/animal-transport-logistics.txt (deflated 86%)
  adding: classified2/com/animal-veterinary-services.txt (deflated 87%)
  adding: classified2/com/architecture-design-services.txt (deflated 87%)
  adding: classified2/com/artistic-design-services.txt (deflated 87%)
  adding: classified2/com/asset-property-inspection-verification.txt (deflated 88%)
  adding: classified2/com/biomedi

#Not needed

In [ ]:
#installing pypy3
!sudo add-apt-repository ppa:pypy/ppa
!sudo apt update
!sudo apt install pypy3
!pypy3 get-pip.py


 The latest release of PyPy packaged for stable Ubuntu releases.

This PPA also contains builds of CFFI for cPython - python-cffi.
 More info: https://launchpad.net/~pypy/+archive/ubuntu/ppa
Press [ENTER] to continue or Ctrl-c to cancel adding it.

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.9 kB]
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.u

In [ ]:
# !pypy3 -m pip install requests
# !pypy3 -m pip install beautifulsoup4
!pypy3 -m pip install gensim
#!pypy3 -m pip install pandas


distutils: /usr/lib/pypy3/../../local/lib/pypy3.7/dist-packages
sysconfig: /usr/lib/pypy3/site-packages
distutils: /usr/lib/pypy3/../../local/lib/pypy3.7/dist-packages
sysconfig: /usr/lib/pypy3/site-packages
distutils: /usr/lib/pypy3/../../local/include
sysconfig: /usr/lib/pypy3/include/UNKNOWN
distutils: /usr/lib/pypy3/../../local/bin
sysconfig: /usr/lib/pypy3/bin
distutils: /usr/lib/pypy3/../../local
sysconfig: /usr/lib/pypy3
user = False
home = None
root = None
prefix = None
  Using cached gensim-4.0.1.tar.gz (23.1 MB)
  Using cached numpy-1.21.1-pp37-pypy37_pp73-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.2 MB)
  Using cached scipy-1.7.1.tar.gz (36.1 MB)
  distutils: /tmp/pip-build-env-9lw_wv0_/normal/site-packages
  sysconfig: /tmp/local/lib/pypy3.7/lib-python
  distutils: /tmp/pip-build-env-9lw_wv0_/normal/site-packages
  sysconfig: /tmp/local/lib/pypy3.7/lib-python
  user = False
  home = None
  root = None
  prefix = '/tmp/pip-build-env-9lw_wv0_/normal'
  distutils: /tmp

In [ ]:
#we load the classification excel file with pandas
#We load the alphabet sheet which has the data we need to populate the game and automatically analyze sounds and shapes of letters
import pandas as pd

xls = pd.ExcelFile('290721.xlsx')
categories_sheet=pd.read_excel(xls, 'categories',keep_default_na=False)
parents_sheet=pd.read_excel(xls, 'parents',keep_default_na=False)

print(categories_sheet.keys())
print(xls.sheet_names)

keyword_dict={}
for row in categories_sheet.iterrows():
  cur_obj={}
  row_dict=row[1].to_dict()
  cat_name=row_dict["Category"]
  if cat_name=="": continue
  keywords=row_dict["Keywords"]
  parent1=row_dict["Parent1"]
  alias1=row_dict["Alias1"]
  parent2=row_dict["Parent2"]
  alias2=row_dict["Alias2"]
  parent3=row_dict["Parent3"]
  alias3=row_dict["Alias3"]
  print(cat_name, "=", keywords)

  # key=row_dict["arabic_script_extended"]
  # letter_ar=row_dict["arabic_script"]


Index(['#', 'Category', 'Parent1', 'Alias1', 'Parent2', 'Alias2', 'Parent3',
       'Alias3', 'Keywords', 'Accuracy of First Page (out of 20)',
       'Accuracy of First Page (%)', 'Action (Select)', 'Action Complete?',
       'Location Info Important to Buyers?',
       'Action after next run (if applicable)', 'Remarks', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')
['parents', 'categories', 'Parked - July 2021']
Accounting Services = Accounting, Bookkeeping, Tax, Records, Returns, Payroll, Lodgement, Statement
Buses & Vans = Bus, Van, Coach, Minibus, Mini, Bus, Sales, Rental, Leasing, Hire, maintenance
Trucks & Utes = Truck, Ute, Sales, Rental, Leasing, Hire, maintenance
Heavy Vehicle Maintenance = Truck, Bus, Trailer, Repairs, Maintenance, Diesel, Heavy, Fleet
Earth Moving Equipment = Earthmoving, Bobcat, Excavator, Loader, Trencher, Dumper, Sales, Rental, Leasing, Hire, maintenance, Backhoe
Financial Audit & Compliance Services = Audit, Compliance, Financial, Money, R

In [ ]:
#4- And now for the main step - caching the content from the list of URLs
#https://www.reddit.com/r/learnpython/comments/8b4q6f/scraping_multiple_web_pages_simultaneously/
#now we adapt it
import concurrent.futures
import urllib.request
from utils.extraction_utils import *

def gen_url_list(fname,start_i=0,n_pages=50,exclude_dict={}):
  fopen=open(fname)
  for li,line in enumerate(fopen):
    if line=="": 
      fopen.close()
      return
    #if li>=20: break
    
    if li<start_i: continue
    if li>=start_i+n_pages: break
    if li%10000==0: print("file is now at:", li)
    cur_url=line.strip("\n\r")
    if exclude_dict.get(cur_url)==True: continue
    yield cur_url
  fopen.close()

crawl_dir="crawl"
run_dir="aug21" #aug21_au
main_dir_path=os.path.join(crawl_dir,run_dir)

url_list_fname="all_urls.txt"
cached_content_fname="cached_all.txt"

cached_url_dict={}
url_list_fpath=os.path.join(main_dir_path,url_list_fname) #"all_au_urls.txt"
cached_content_fpath=os.path.join(main_dir_path,cached_content_fname) #"cached_all_au.txt"

if os.path.exists(cached_content_fpath):
  cached_fopen=open(cached_content_fpath)
  for i,f in enumerate(cached_fopen):
    # f_split=f.split("<br>")
    url=f.split("<br>")[0]
    if not url.startswith("http"): continue
    cached_url_dict[url]=True
  cached_fopen.close()

n_urls=file_len(url_list_fpath)
print("number of urls to process:",n_urls)

batch_size=1000
for i_ in range(0,n_urls,batch_size): #the end number should be the size of the file being read
  print("current i",i_)
  URLS=gen_url_list(url_list_fpath,i_,batch_size,cached_url_dict)

  # We can use a with statement to ensure threads are cleaned up promptly
  content_list=[]
  with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
      # Start the load operations and mark each future with its URL
      #future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
      future_to_url = {executor.submit(get_content, url): url for url in URLS if cached_url_dict.get(url)!=True}
      for future in concurrent.futures.as_completed(future_to_url):
          url = future_to_url[future]
          try:
              data = future.result()
          except Exception as exc:
              #print('%r generated an exception: %s' % (url, exc))
              pass
          else:
              #print('%r page is %d bytes' % (url, len(data)))
              content_list.append(data)
              #if len(content_list)%500==0: print("current content list length", len(content_list))
              #print(len(content_list))
  cached_content_fopen=open(cached_content_fpath,"a")
  
  for cl in content_list:
    cached_content_fopen.write(cl+"\n")
  cached_content_fopen.close()

/content/drive/My Drive/KM/utils/extraction_utils.py:49: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 49 of the file /content/drive/My Drive/KM/utils/extraction_utils.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup=BeautifulSoup(self.html_content)


number of urls to process: 553586
current i 0
file is now at: 0
current i 1000
current i 2000
current i 3000
current i 4000
current i 5000
current i 6000
current i 7000
current i 8000
current i 9000
current i 10000
file is now at: 10000
current i 11000
current i 12000
current i 13000
current i 14000
current i 15000
current i 16000
current i 17000
current i 18000
current i 19000
current i 20000
file is now at: 20000
current i 21000
current i 22000
current i 23000
current i 24000
current i 25000
current i 26000
current i 27000
current i 28000
current i 29000
current i 30000
file is now at: 30000
current i 31000
current i 32000
current i 33000
current i 34000
current i 35000
current i 36000
current i 37000
current i 38000
current i 39000
current i 40000
file is now at: 40000
current i 41000
current i 42000
current i 43000
current i 44000
current i 45000
current i 46000
current i 47000
current i 48000
current i 49000
current i 50000
file is now at: 50000
current i 51000
current i 52000
cur

In [ ]:
#Now run the caching script on the small data with 60 workers and a batch of 10000
!chmod +x utils/caching_utils.py
!utils/caching_utils.py #with 10000 batch - 60 workers
#completed in 433.3331551551819 seconds

number of urls to process: 553586
completed in 10.773498058319092 seconds


In [ ]:
!chmod +x utils/caching_utils.py
!utils/caching_utils.py #with 1000 batch - 30 workers
#completed in 588.3326060771942 seconds

number of urls to process: 7519
current i 0 time: 0.0
current i 1000 time: 68.52
current i 2000 time: 87.21
current i 3000 time: 78.14
current i 4000 time: 72.93
current i 5000 time: 78.81
current i 6000 time: 77.6
current i 7000 time: 73.02
completed in 588.3326060771942 seconds


In [ ]:
#with 1000 batch - 50 workers
!utils/caching_utils.py
#completed in 456.4794147014618 seconds

number of urls to process: 7519
current i 0 time: 0.0
current i 1000 time: 50.1
current i 2000 time: 71.2
current i 3000 time: 56.57
current i 4000 time: 57.38
current i 5000 time: 57.3
current i 6000 time: 57.81
current i 7000 time: 54.67
completed in 456.4794147014618 seconds


In [ ]:
#with 1000 batch - 100 workers
!utils/caching_utils.py
#completed in 463.1788227558136 seconds

number of urls to process: 7519
current i 0 time: 0.0
current i 1000 time: 52.63
current i 2000 time: 69.95
current i 3000 time: 57.14
current i 4000 time: 56.31
current i 5000 time: 54.81
current i 6000 time: 57.28
current i 7000 time: 70.12
completed in 463.1788227558136 seconds


In [ ]:
#with 1000 batch - 40 workers
!utils/caching_utils.py
#completed in 507.22553849220276 seconds

number of urls to process: 7519
current i 0 time: 0.0
current i 1000 time: 57.18
current i 2000 time: 72.82
current i 3000 time: 63.35
current i 4000 time: 75.02
current i 5000 time: 63.45
current i 6000 time: 64.5
current i 7000 time: 58.95
completed in 507.22553849220276 seconds


In [ ]:
#with 1000 batch - 60 workers
!utils/caching_utils.py
#completed in 448.39253640174866 seconds

number of urls to process: 7519
current i 0 time: 0.0
current i 1000 time: 49.13
current i 2000 time: 64.02
current i 3000 time: 55.23
current i 4000 time: 66.33
current i 5000 time: 54.71
current i 6000 time: 55.32
current i 7000 time: 50.48
completed in 448.39253640174866 seconds


In [ ]:
!pypy3 utils/caching_utils.py

number of urls to process: 7519
current i 0 time: 0.0
/content/drive/MyDrive/KM/utils/extraction_utils.py:49: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 49 of the file /content/drive/MyDrive/KM/utils/extraction_utils.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup=BeautifulSoup(self.html_content)
current i 1000 time: 69.56
current i 2000 time: 81.82
current i 3000 time: 76.69
current i 4000 time: 76.78
current i 5000 time: 80.28
current i 6000 time: 77.0
current i 7000 time: 76.4
completed in 580.5658779144287 seconds


In [ ]:
# !tail -5 crawl/aug21/cached_all.txt
# !wc crawl/aug21/cached_all.txt
fpath="crawl/aug21/cached_all.txt"
url_dict={}
fopen=open(fpath)
for i,f in enumerate(fopen):
  # f_split=f.split("<br>")
  url=f.split("<br>")[0]
  if not url.startswith("http"): continue
  url_dict[url]=True
fopen.close()
print(len(url_dict.keys()) )

325761


In [ ]:
# !pypy3 -m pip install requests
# !pypy3 -m pip install beautifulsoup4
# !pypy3 -m pip install gensim
!pypy3 -m pip install futures

distutils: /usr/lib/pypy3/../../local/lib/pypy3.7/dist-packages
sysconfig: /usr/lib/pypy3/site-packages
distutils: /usr/lib/pypy3/../../local/lib/pypy3.7/dist-packages
sysconfig: /usr/lib/pypy3/site-packages
distutils: /usr/lib/pypy3/../../local/include
sysconfig: /usr/lib/pypy3/include/UNKNOWN
distutils: /usr/lib/pypy3/../../local/bin
sysconfig: /usr/lib/pypy3/bin
distutils: /usr/lib/pypy3/../../local
sysconfig: /usr/lib/pypy3
user = False
home = None
root = None
prefix = None
  distutils: /usr/lib/pypy3/../../local/include
  sysconfig: /usr/lib/pypy3/include/futures


In [ ]:
fname="crawl/aug21/all_urls.txt"
fopen=open(fname)
for i,f in enumerate(fopen):
  if i>100: break
  print([f])
  # url=f.split("<br>")[0]
  # if not url.startswith("http"):  print([f])
fopen  
fopen.close()


['https://embryology.med.unsw.edu.au\n']
['https://emed.med.unsw.edu.au\n']
['https://ethics.med.unsw.edu.au\n']
['https://iser.med.unsw.edu.au\n']
['https://medicalsciences.med.unsw.edu.au\n']
['https://medprogram.med.unsw.edu.au\n']
['https://ndarc.med.unsw.edu.au\n']
['http://php.med.unsw.edu.au\n']
['https://powcs.med.unsw.edu.au\n']
['https://rmlab.med.unsw.edu.au\n']
['https://sims.med.unsw.edu.au\n']
['https://sph.med.unsw.edu.au\n']
['https://sphcm.med.unsw.edu.au\n']
['https://staff.med.unsw.edu.au\n']
['https://stgcs.med.unsw.edu.au\n']
['https://stvcs.med.unsw.edu.au\n']
['https://swscs.med.unsw.edu.au\n']
['https://teachingtools.med.unsw.edu.au\n']
['https://wch.med.unsw.edu.au\n']
['http://web.med.unsw.edu.au\n']
['https://membrane.unsw.edu.au\n']
['https://mentoring.unsw.edu.au\n']
['https://meridian180.unsw.edu.au\n']
['http://merlin.unsw.edu.au\n']
['https://mhdcd.unsw.edu.au\n']
['http://mining.unsw.edu.au\n']
['http://mm2015.unsw.edu.au\n']
['https://music.unsw.edu.au

In [ ]:
!wc -l crawl/aug21/gov.au.txt
!wc -l crawl/aug21/cached_gov11.txt

7519 crawl/aug21/gov.au.txt
7519 crawl/aug21/cached_gov11.txt
